<a href="https://colab.research.google.com/github/Emman0060/Assembler/blob/main/Game_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#F535780 2025-01-02
#Has all functions and makes it more practial and easier to read code
#ChatHPT has been used to help way of producing this such as using a switch case to produce my inventory pruning method (Removalsuggestions)
#Geek4Geeks has also been used so I can see the syntax of some commands.

%%writefile database.py

import subscriptionManager

def FindGameByID(GameID):
  #Allows you to find infomaton about a game from its id and tells whether is video game or board game
  with open("Video_Game_Info.txt", "r") as file:
    for line in file:
      Line = line
      ListOfItems = Line.strip().split("|")
      if (ListOfItems[0] == GameID):
        return "IsVidedGame", Line

  with open("Board_Game_Info.txt", "r") as file:
    for line in file:
      Line = line
      ListOfItems = Line.strip().split("|")
      if (ListOfItems[0] == GameID):
        return "IsBoardGame", Line

  return None,None


def ShowGameRecords(NameOfFile): #2025-12-25
#return information about whole file by inputting its name
  record = []

  if (NameOfFile == "Video_Game_Info.txt"):
    with open("Video_Game_Info.txt", "r") as f:
      for line in f:
        record.append(line)
      return record

  elif (NameOfFile == "Board_Game_Info.txt"):
    with open("Board_Game_Info.txt", "r") as f:
      for line in f:
        record.append(line)
      return record

  elif (NameOfFile == "Rental.txt"):
    with open("Rental.txt", "r") as f:
      for line in f:
        record.append(line)
      return record

  elif (NameOfFile == "Booking.txt"):
    with open("Rental.txt", "r") as f:
      for line in f:
        record.append(line)
      return record

  elif (NameOfFile == "Game_Feedback.txt"):
    with open("Rental.txt", "r") as f:
      for line in f:
        record.append(line)
      return record

  else:
    return []


def IsGameRented(GameID):
  #Check if game is rented with gameid by checking game id and that returndate is blank
  Records = ShowGameRecords("Rental.txt")
  for lines in Records:
    Line = lines.split("|")
    if (lines.startswith("GameID")):
      continue

    if (Line[0].strip() == GameID and Line[2].strip() == ""):
      return True
  return False

def AddRecords(NameOfFile, RecordInfo):
  #add record to a fileby filename and information to add
  file = open(NameOfFile, "a")
  file.write(RecordInfo + "\n")
  file.close()

def RentAGame():
  #Rent a game, checks that not over limit, subscription is valid, and game is not rented out
  GameID = input("Enter the game id: ")
  RentalFromDate = input("Enter date in correct format: ")
  CustomerID = input("Enter your id: ").lower()
  RentalToDate = ""
  RentLimit = subscriptionManager.get_rental_limit(CustomerID)

  currentRented = 0
  for line in ShowGameRecords("Rental.txt"):
    part = line.split("|")
    if (part[3] == CustomerID and part[2] == ""):
      currentRented += 1

  if (currentRented >= RentLimit):
    print("Limit reached")
    return


  if (subscriptionManager.check_subscription(CustomerID) == False):
    print("No subscription")
    return

  if (IsGameRented(GameID)):
    print("Game is rented out ")
    return

  VideoOrBoardGame, Info = FindGameByID(GameID)

  if (VideoOrBoardGame is None):
    print("Game not found")
    return None,None

  line = GameID + "|" + RentalFromDate + "|" + RentalToDate + "|" + CustomerID + "|"

  AddRecords("Rental.txt", line)

  print("Record added!")

def ReturnGame(GameID, RentalToDate):
  #reutrn a game, allows you to leave rating and comments too, uses gameid and returned date
  IsRentalActive = False
  GameType, Info = FindGameByID(GameID)
  NewRecords = []
  if (GameType is None):
    print("Game not found")
    return False

  AllInfo = ShowGameRecords("Rental.txt")

  for line in AllInfo:
    parts = line.split("|")
    if (parts[0] == GameID and parts[2] == "" and IsRentalActive == False):
      parts[2] = RentalToDate
      IsRentalActive = True

  NewRecordLine = parts[0] + "|" + parts[1] + "|" + parts[2] + "|" + parts[3] + "|"
  NewRecords.append(NewRecordLine)


  if (IsRentalActive == False):
    print("Game has not been rented")
    return

  file = open("Rental.txt", "w")
  file.write("GameID|RentalDate|ReturnDate|RentalCustomerID|\n")
  for line in NewRecords:
      file.write(line + "\n")
  file.close()


  rating = input("Out of 5 stars how did you find the game: ")
  comments = input("Enter any additional comments: ")

  Feedback = GameID + "," + rating + "," + comments + "|"

  AddRecords("Game_Feedback.txt", Feedback)

  print("Game returned")


def IsValidtime(Time):
  #makes sure valid time is entered
  if (Time == "2pm" or Time == "6pm"):
    return True
  return False

def ValidGuestsNO(NOOfGuests):
  #checks valid guest number
  if (NOOfGuests.isdigit() == False):
    return False

  if (int(NOOfGuests) >= 0 and int(NOOfGuests) <= 3):
    return True
  return False

def BookSession():
  #user can book session, few checks then add to booking txt
   userID = input("Enter ID")
   BookingDate = input("Enter booking date")
   Time = input("Enter time slot (2pm or 6pm): ")
   NOGuests = input("Number no. guests ")
   Guests = 0

   if(IsValidtime(Time) == False):
    print("Incorrect time")
    return

   if (ValidGuestsNO(NOGuests) == False):
    print("Invalid numer of guests")
    return

   if (NOGuests.isdigit() == False):
     print("Number of guests must be integer")
     return

   Guests = int(NOGuests)

   if (subscriptionManager.check_subscription(userID) == False):
    print("No subscription")
    return

   BookingLine = userID + "|" + BookingDate + "|" + Time + "|" + str(Guests) + "|"
   AddRecords("Booking.txt", BookingLine)
   print("Booking complete")


def RemovalSuggestion():
  #check if id in is rented if not rented for more than 3 times then will be reccommended to remove
  record = ShowGameRecords("Rental.txt")

  count = {}

  for lines in record:
    part = lines.split("|")
    GameID = part[0]

    for i in count:
      if (i == GameID):
        count[GameID] += 1
      else:
        count[GameID] = 1


  SuggestionMade = False

  for i in count:
    if (count[i] < 2):
      print(i + " should be removed.")
      SuggestionMade = True

  if (SuggestionMade == False):
    print("No suggestions")

  return

Overwriting database.py


In [37]:
#F535780 2025-01-05
#This is the main menu which the user will interact with.

%%writefile GameMenu.py

import importlib, database
importlib.reload(database)

def menu():
  #switch case for different options then call method
  while True:
    print("")
    print("*** Welcome to Emans' Game Store! ***")
    print("Type 1 to rent a game")
    print("Type 2 to return a game")
    print("Type 3 to book a session")
    print("Type 4 to view all video games")
    print("Type 5 to view all board games")
    print("Type 6 for removal suggestions")
    print("Type 7 to exit")

    entry = input("Enter choice: ")
    print("")
    match entry:
      case "1":
        database.RentAGame()
      case "2":
        GameID = input("Enter game ID: ")
        ReturnToDate = input("Enter return date: ")
        database.ReturnGame(GameID, ReturnToDate)
      case "3":
        database.BookSession()
      case "4":
        with open("Video_Game_Info.txt", "r") as file:
          for line in file:
            print(line)
      case "5":
        with open("Board_Game_Info.txt", "r") as file:
          for line in file:
            print(line)
      case "6":
        database.RemovalSuggestion()
      case "7":
        print("See you next time!")
        return False

menu()

Overwriting GameMenu.py


In [36]:
#F535780 2025-01-06
#Calls main menu for user

#Security issues
'''
Invalid inputs can cause the program to crash
Files are not encrypted so people may be able to see when others have booked
Booking can be made by anyone as there is no access levels
No error handling of files if they do not can cause crashes
Anyone can acces file informaton without permission
'''

#Start program



menu()


*** Welcome to Emans' Game Store! ***
Type 1 to rent a game
Type 2 to return a game
Type 3 to book a session
Type 4 to view all video games
Type 5 to view all board games
Type 6 for removal suggestions
Type 7 to exit


KeyboardInterrupt: Interrupted by user